# Introdução

## Importação das bibliotecas necessarias

In [3]:
import requests
import pandas as pd
import os
import json
import sys
import matplotlib.pyplot as plt
import funcoes_comuns as fc
from importlib import reload
reload(fc)

# Dataframe base de Código do município e Ano

In [4]:
base_df = fc.get_tb_municipios()
base_df.head()

,CODIGO_MUNICIPIO,ano_nasc,CODMUNRES
0,1100015,2012,110001
1,1100015,2013,110001
2,1100015,2014,110001
3,1100015,2015,110001
4,1100015,2016,110001


# Nascidos vivos de mulheres de 2012 a 2020

In [6]:
%time
df_t_nasc_vivos = fc.build_df_by_sql_query("SELECT CODMUNRES, ano_nasc, count(*) as TOTAL_DE_NASCIDOS_VIVOS \
                    FROM \"datasus-sinasc\" \
                    WHERE ano_nasc>=2012 AND \
                    ano_nasc<=2020 \
                    GROUP BY CODMUNRES, ano_nasc")

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 8.34 µs
...................................................(y)
Record rows length : 50268


# Número de nascidos vivos de mulheres com menos de 20 anos

In [7]:
%time
df_nvm_menor_20 = fc.build_df_by_sql_query("SELECT CODMUNRES, ano_nasc, count(*) as NVM_MENOR_QUE_20 \
                    FROM \"datasus-sinasc\" \
                    WHERE ano_nasc>=2012 AND \
                    ano_nasc<=2020 AND \
                    IDADEMAE<20 \
                    GROUP BY CODMUNRES, ano_nasc")

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 0 ns
..................................................(y)
Record rows length : 49810


# Número de mulheres com mais de três partos anteriores

In [8]:
%time
df_mais_de_3parto = fc.build_df_by_sql_query("SELECT CODMUNRES, ano_nasc, count(*) as MULHERES_COM_MAIS_DE_TRES_PARTOS_ANTERIORES \
                    FROM \"datasus-sinasc\" \
                    WHERE (ano_nasc>=2012 AND \
                    ano_nasc<=2020) AND \
                    (QTDPARTNOR > 3 \
                    OR (QTDPARTNOR > 2 AND QTDPARTCES > 0) \
                    OR (QTDPARTNOR > 1 AND QTDPARTCES > 1) \
                    OR (QTDPARTNOR > 0 AND QTDPARTCES > 2) \
                    OR QTDPARTCES > 3) \
                    GROUP BY CODMUNRES, ano_nasc")

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 0 ns
..............................................(y)
Record rows length : 45157


# População feminina de 10 a 20 anos

In [9]:
#pop_fem_menor_20 = pd.read_csv('../../csv_files/Alternativas/indicadores_bloco2_planejamento_reprodutivo_2012-2020.csv')
pop_fem_menor_20 = []

for i in range(9):
    fc.dot_print()
    pop_fem_menor_20.extend(fc.fetch_pop_fem_from_tabnet(str(12+i).zfill(2), '20'))
print('(y)')

df_pop_fem_menor_20 = pd.DataFrame(pop_fem_menor_20, columns = ['CODMUNRES', 'ANO', 'POP_FEMININA_10_A_19'])
df_pop_fem_menor_20 = df_pop_fem_menor_20.rename(columns={"ANO": "ano_nasc"})
df_pop_fem_menor_20

.........(y)


,CODMUNRES,ano_nasc,POP_FEMININA_10_A_19
0,110001,2012,2256
1,110002,2012,9407
2,110003,2012,523
3,110004,2012,7515
4,110005,2012,1564
...,...,...,...
50125,522200,2020,930
50126,522205,2020,609
50127,522220,2020,420
50128,522230,2020,354


# População Feminina de 10 a 49 anos 

In [10]:
pop_fem = []

for i in range(9):
    fc.dot_print()
    pop_fem.extend(fc.fetch_pop_fem_from_tabnet(str(12+i).zfill(2), '49'))
print('(y)')

df_pop_fem = pd.DataFrame(pop_fem, columns = ['CODMUNRES', 'ano_nasc', 'POP_FEM_10_49'])

#df_pop_fem = pd.read_csv('../../csv_files/Alternativas/indicadores_bloco2_planejamento_reprodutivo_2012-2020.csv')

.........(y)


# Internações por abortamento

In [12]:
dfs = []
for i in range(9):
    fc.dot_print()
    dfs.append(pd.read_csv('./taxa_de_aborto_inseguro/61e62_brasil_'+str(2012+i)+'.csv'))

fetch_mun_df = pd.read_excel("../../csv_files/tb_municipios.xlsx")
mun_df = pd.DataFrame([], columns = ['CODIGO_MUNICIPIO'])
mun_df['CODIGO_MUNICIPIO'] = fetch_mun_df['codigo_municipio_completo']
mun_df['CODIGO_MUNICIPIO'] = mun_df['CODIGO_MUNICIPIO'].astype(str)
mun_df['CODMUNRES'] = mun_df['CODIGO_MUNICIPIO'].replace(r'.$', '', regex=True)
    
for i in range(len(dfs)):
    fc.dot_print()
    dfs[i]['ANO'] = 2012+i
    dfs[i] = dfs[i].dropna(subset=['total_mulheres'])
    dfs[i] = dfs[i].reset_index(drop=True)
    dfs[i]['CODMUNRES'] = mun_df['CODMUNRES'].astype(int)

df_acumulados = pd.concat(dfs, axis=0, ignore_index=True)
df_acumulados = df_acumulados.iloc[:, [10, 9, 0, 1, 2, 3, 4, 5, 6, 7, 8]]
df_acumulados['CODMUNRES'] = df_acumulados['CODMUNRES'].astype(int)
df_acumulados = df_acumulados.fillna(0)
df_acumulados = df_acumulados.sort_values(['CODMUNRES', 'ANO'])
df_acumulados = df_acumulados.reset_index(drop=True)

print('(y)')

..................(y)


# População feminina com plano de saúde

In [14]:
df_xlsx = pd.read_excel("analise_da_cobertura_sus_-_com_substituicoes_27_10_2022.xlsx", sheet_name='cobertura sus por substituicao')
df_aux = df_xlsx.iloc[:,[0,24,25,26,27,28,29,30,31,32]]
df_aux = df_aux.dropna(subset=['codmun'])
df_ano = pd.DataFrame()
df_codmunres = pd.DataFrame()

lista = []
codmunres = []
transposto = []
fc.dot_print()
for row in range(len(df_aux.index)):
    codmunres.append(df_aux.iloc[row, 0])
    lista.append(df_aux.iloc[row, 1:10].values.tolist())
fc.dot_print()
for i in range(len(lista)):
    for j in range(len(lista[i])):
        transposto.append(lista[i][j])
fc.dot_print()

df_codmunres['CODMUNRES'] = codmunres
df_codmunres['CODMUNRES'] = df_codmunres['CODMUNRES'].astype(int)

df_ano['ano_nasc'] = [2012,2013,2014,2015,2016,2017,2018,2019,2020]
df_cob_sus = pd.merge(df_codmunres.assign(key=1), df_ano.assign(key=1),on='key').drop('key', axis=1)
df_cob_sus['COBERTURA_SUS'] = transposto


print('(y)')

...(y)


In [15]:
df_tx_abortos_inseguros = pd.DataFrame([], columns = ['CODMUNRES', 'ano_nasc', 'INTERNACOES_ABORTO', 'POP_FEM_10_49_COM_PLANO_SAUDE'])

df_tx_abortos_inseguros['CODMUNRES'] = df_acumulados['CODMUNRES']
df_tx_abortos_inseguros['ano_nasc'] = df_acumulados['ANO']
df_tx_abortos_inseguros['INTERNACOES_ABORTO'] = df_acumulados['numero_internacoes_por_abortamento'].astype(int)
df_tx_abortos_inseguros['INTERNACOES_ABORTO'] = df_acumulados['numero_internacoes_por_abortamento'].astype(int)
df_tx_abortos_inseguros['POP_FEM_10_49'] = df_pop_fem['POP_FEM_10_49']
#df_tx_abortos_inseguros['POP_FEM_10_49_COM_PLANO_SAUDE'] = (df_acumulados['cobertura_saude_suplementar']*df_acumulados['total_mulheres']).astype(int)
#df_tx_abortos_inseguros['POP_FEM_10_49_COM_PLANO_SAUDE'] = (df_acumulados['cobertura_saude_suplementar']*df_pop_fem['POP_FEM_10_49'].astype(int)).astype(int)
df_tx_abortos_inseguros['POP_FEM_10_49_COM_PLANO_SAUDE'] = ((1.0-df_cob_sus['COBERTURA_SUS'])*df_pop_fem['POP_FEM_10_49'].astype(int)).astype(int)

# Finalização

In [17]:
# Conversão do CODMUNRES para inteiro, para que seja possível a junção
base_df['CODMUNRES']=base_df['CODMUNRES'].astype(int)
df_t_nasc_vivos['CODMUNRES']=df_t_nasc_vivos['CODMUNRES'].astype(int)
df_nvm_menor_20['CODMUNRES']=df_nvm_menor_20['CODMUNRES'].astype(int)
df_pop_fem_menor_20['CODMUNRES']=df_pop_fem_menor_20['CODMUNRES'].astype(int)
df_pop_fem_menor_20['ano_nasc']=df_pop_fem_menor_20['ano_nasc'].astype(int)
df_mais_de_3parto['CODMUNRES']=df_mais_de_3parto['CODMUNRES'].astype(int)
df_pop_fem['CODMUNRES']=df_pop_fem['CODMUNRES'].astype(int)
#df_pop_fem['ano_nasc']=df_pop_fem['ANO'].astype(int)
df_tx_abortos_inseguros['CODMUNRES']=df_tx_abortos_inseguros['CODMUNRES'].astype(int)

# Junção em um dataframe final
df_final = pd.merge(base_df, df_t_nasc_vivos, how='left', on=['CODMUNRES','ano_nasc'])
df_final = pd.merge(df_final, df_nvm_menor_20, how='left', on=['CODMUNRES','ano_nasc'])
df_final = pd.merge(df_final, df_pop_fem_menor_20, how='left', on=['CODMUNRES','ano_nasc'])
df_final = pd.merge(df_final, df_mais_de_3parto, how='left', on=['CODMUNRES','ano_nasc'])
#df_final = pd.merge(df_final, df_pop_fem, how='left', on=['CODMUNRES','ano_nasc'])
df_final = pd.merge(df_final, df_tx_abortos_inseguros, how='left', on=['CODMUNRES','ano_nasc'])
# Sífilis congênita

# Preencher NaN com zeros
df_final = df_final.fillna(0)
df_final.rename(columns={'ano_nasc': 'ANO'},
               inplace=True)

del df_final['CODIGO_MUNICIPIO']

[i in range(len(df_final.columns.values.tolist()))]
aux = [i for i in range(len(df_final.columns.values.tolist()))]
aux[0] = 1
aux[1] = 0
df_final = df_final.iloc[:, aux]

In [18]:
df_final

,CODMUNRES,ANO,TOTAL_DE_NASCIDOS_VIVOS,NVM_MENOR_QUE_20,POP_FEMININA_10_A_19,MULHERES_COM_MAIS_DE_TRES_PARTOS_ANTERIORES,INTERNACOES_ABORTO,POP_FEM_10_49_COM_PLANO_SAUDE,POP_FEM_10_49
0,110001,2012,386.0,76.0,2256,20.0,14,624,7810
1,110001,2013,382.0,79.0,2162,19.0,12,2607,32588
2,110001,2014,341.0,69.0,2065,19.0,17,149,1873
3,110001,2015,373.0,74.0,1966,15.0,8,2209,27621
4,110001,2016,326.0,58.0,1900,19.0,13,454,5676
...,...,...,...,...,...,...,...,...,...
50125,530010,2016,43340.0,5159.0,228456,1575.0,105,1322,4265
50126,530010,2017,44568.0,4959.0,227457,1712.0,248,774,2580
50127,530010,2018,44195.0,4734.0,225494,1611.0,1504,490,1691
50128,530010,2019,42422.0,4361.0,222842,1635.0,1532,482,1607


In [19]:
df_final.columns.values.tolist()

['CODMUNRES',
 'ANO',
 'TOTAL_DE_NASCIDOS_VIVOS',
 'NVM_MENOR_QUE_20',
 'POP_FEMININA_10_A_19',
 'MULHERES_COM_MAIS_DE_TRES_PARTOS_ANTERIORES',
 'INTERNACOES_ABORTO',
 'POP_FEM_10_49_COM_PLANO_SAUDE',
 'POP_FEM_10_49']

In [20]:
df_final.to_csv('./indicadores_bloco2_planejamento_reprodutivo_2012-2020.csv',index=False)